In [ ]:
import astropy.coordinates as coord
from astropy.io import fits
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from gala.mpl_style import hesperia_r

from totoro.data import datasets
from totoro.config import galcen_frame
from totoro.abundance_helpers import elem_to_label

In [ ]:
for name, d in datasets.items():
    print(name, len(d))

In [ ]:
vstep = 1.5
zstep = 50 / 1e3
vzz_bins = (np.arange(-82.5, 82.5+1e-3, vstep),
            np.arange(-1.5, 1.5+1e-3, zstep))

In [ ]:
from matplotlib.gridspec import GridSpec

def make_dataset_figure(d, xname, yname,
                        cmap='hesperia_r', 
                        xy_size=2.6*u.kpc, xy_step=25*u.pc):

    fig = plt.figure(figsize=(12, 10),
                     constrained_layout=True)

    gs = GridSpec(nrows=2, ncols=2, figure=fig)
    ax1 = fig.add_subplot(gs[0, :])
    ax2 = fig.add_subplot(gs[1, 0])
    ax3 = fig.add_subplot(gs[1, 1])

    bins = (np.arange(-2., 0.75, 0.01),
            np.arange(-0.18, 0.4, 0.005))

    H, xe, ye = np.histogram2d(d.get_elem_ratio(xname), 
                               d.get_elem_ratio(yname), 
                               bins=bins)
    norm = mpl.colors.LogNorm() #vmin=1, vmax=10**np.nanpercentile(np.log10(H), 99))
    cs = ax1.pcolormesh(xe, ye, H.T, norm=norm, cmap=cmap, rasterized=True)

    ax1.set_xlim(-1.5, 0.7)
    ax1.set_ylim(-0.12, 0.32)
    ax1.set_xlabel(f'metallicity, {elem_to_label(xname)}')
    ax1.set_ylabel(f'alpha abundance, {elem_to_label(yname)}')

    # ---
    
    c = d.c
    x = c.galactocentric.x.to_value(u.kpc)
    y = c.galactocentric.y.to_value(u.kpc)
    z = c.galactocentric.z.to_value(u.kpc)
    vz = c.galactocentric.v_z.to_value(u.km / u.s)

    xy_size = xy_size.to_value(u.kpc)
    xy_step = xy_step.to_value(u.kpc)
    bins = (np.arange(-8.122 - xy_size, -8.122 + xy_size + 1e-3, xy_step),
            np.arange(-xy_size, xy_size + 1e-3, xy_step))
    H, xe, ye = np.histogram2d(x, y, bins=bins)
    ax2.pcolormesh(xe, ye, H.T, norm=norm, cmap=cmap, rasterized=True)

    ax2.set_xlabel('Galactocentric $x$ [kpc]')
    ax2.set_ylabel('Galactocentric $y$ [kpc]')

    H, xe, ye = np.histogram2d(vz, z, bins=vzz_bins)
    ax3.pcolormesh(xe, ye, H.T, norm=norm, cmap=cmap, rasterized=True)

    ax3.set_xlabel(f'Galactocentric $v_z$ [{u.km/u.s:latex_inline}]')
    ax3.set_ylabel('Galactocentric $z$ [kpc]')

    cb = fig.colorbar(cs, ax=ax1, aspect=30)
    cb.set_label('$N$ stars')

    fig.set_facecolor('w')

    return fig, [ax1, ax2, ax3]

In [ ]:
for name, d in datasets.items():
    if name.lower().startswith('galah'):
        fig, axes = make_dataset_figure(d, 'FE_H', 'ALPHA_FE')
    else:
        fig, axes = make_dataset_figure(d, 'M_H', 'ALPHA_M')
    fig.suptitle(name, fontsize=22)